In [ ]:
%load_ext lab_black

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

# Parameters

In [ ]:
data_date = "2020-06-01"  # the month that we are computing -> still ran once, but will do all days in the month (i.e. data_date = '2022-04-01' will process the whole of Apr)

presence_reference_date = "2020-01-01"
trips_reference_date = "2020-01-01"

shared_data_dir = "data"  # donde esta la data that can change
static_dir = "static"  # wo ist das static data

aggregates_subdir = (
    "aggregates"  # where art the aggregate data (within shared_data_dir)
)
indicators_subdir = "indicators"  # where art the indicators (within shared_data_dir)

# Filenames of auxiliary files in static_dir
geometry_filename = "admin3.geojson"  # file with shapefile for residents per km2
base_pop_and_growth_rates_filename = "Haiti population base data 1 Jan 2020 by admin3 v2.0 narrow.csv"  # file descibing base pop and growth rates
x_scaling_factors_filename = "pop_scaling_x_factors.csv"

base_pop_column = "est_pop_2020_01"
pop_and_gr_spatial_unit = "pcod"
x_scaling_factors_spatial_unit = "pcod"

metric_crs_epsg = (
    32618  # country specific, what projection to use for metre coordinates
)

In [ ]:
# Convert date parameters to datetime objects here because papermill date parameters will be strings
data_date = pd.to_datetime(data_date)
presence_reference_date = pd.to_datetime(presence_reference_date)
trips_reference_date = pd.to_datetime(trips_reference_date)

# Get full path to data subdirs
aggregates_dir = Path(shared_data_dir) / aggregates_subdir
indicators_dir = Path(shared_data_dir) / indicators_subdir

# Get full path to static files
geometry_filepath = Path(static_dir) / geometry_filename
base_pop_and_growth_rates_filepath = (
    Path(static_dir) / base_pop_and_growth_rates_filename
)
x_scaling_factors_filepath = Path(static_dir) / x_scaling_factors_filename

# What this notebook is doing

In [ ]:
"Creating trips + presence indicators for the month of"

In [ ]:
data_date

In [ ]:
"using "

In [ ]:
base_pop_and_growth_rates_filename

In [ ]:
x_scaling_factors_filename


---

First get everything we need to scale indicators:

## Data required for base indicators x factor scaling

In [ ]:
# Create the indicators subdir if it doesn't already exist
indicators_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
month_1_dates = pd.date_range(
    presence_reference_date,
    presence_reference_date + pd.DateOffset(months=1) - pd.DateOffset(days=1),
)

month_X_dates = pd.date_range(
    data_date, data_date + pd.DateOffset(months=1) - pd.DateOffset(days=1)
)

### presence_subscribers_month_1

In [ ]:
presence_days = []

for date in month_1_dates:
    with (
        Path(aggregates_dir)
        / "presence_trips"
        / f"presence_trips_aggregates_{presence_reference_date.date()}"
        / f"subscriber-counts_nosubset_{date.date()}.csv"
    ) as filepath:
        if Path(filepath).is_file():
            presence_day_X = pd.read_csv(
                Path(aggregates_dir)
                / "presence_trips"
                / f"presence_trips_aggregates_{presence_reference_date.date()}"
                / f"subscriber-counts_nosubset_{date.date()}.csv"
            )
            presence_day_X["date"] = date.date()

            presence_days.append(presence_day_X)

In [ ]:
presence_subs_month_1 = pd.concat(presence_days)

### presence median ref period

In [ ]:
# Ensure each pcod present has a value for every date (filling missing/redacted counts with 0)
presence_subs_month_1_filled = (
    presence_subs_month_1.set_index(["pcod", "date"])
    .unstack(fill_value=0)
    .stack()
    .reset_index()
)

presence_median_per_loc = (
    presence_subs_month_1_filled.groupby("pcod").value.median().reset_index()
)

# Drop pcods with median = 0 (we can't use these for scaling)
presence_median_per_loc = presence_median_per_loc.loc[
    presence_median_per_loc["value"] > 0
]

### presence_subscribers_month_X

In [ ]:
presence_days = []

for date in month_X_dates:
    with (
        Path(aggregates_dir)
        / "presence_trips"
        / f"presence_trips_aggregates_{data_date.date()}"
        / f"subscriber-counts_nosubset_{date.date()}.csv"
    ) as filepath:
        if Path(filepath).is_file():
            presence_day_X = pd.read_csv(
                Path(aggregates_dir)
                / "presence_trips"
                / f"presence_trips_aggregates_{data_date.date()}"
                / f"subscriber-counts_nosubset_{date.date()}.csv"
            )
            presence_day_X["date"] = date.date()

            presence_days.append(presence_day_X)

In [ ]:
presence_subs_month_X = pd.concat(presence_days)

### trips_subscribers_month_1

In [ ]:
trips_days = []

for date in month_1_dates:
    with (
        Path(aggregates_dir)
        / "presence_trips"
        / f"presence_trips_aggregates_{presence_reference_date.date()}"
        / f"consecutive-trips_nosubset_{date.date()}.csv"
    ) as filepath:
        if Path(filepath).is_file():
            trips_day_X = pd.read_csv(
                Path(aggregates_dir)
                / "presence_trips"
                / f"presence_trips_aggregates_{presence_reference_date.date()}"
                / f"consecutive-trips_nosubset_{date.date()}.csv"
            )
            trips_day_X["date"] = date.date()

            trips_days.append(trips_day_X)

In [ ]:
trips_subs_month_1 = pd.concat(trips_days)

### trips_median_ref_period

In [ ]:
# Ensure each pair of pcods present has a value for every date (filling missing/redacted counts with 0)
trips_subs_month_1_filled = (
    trips_subs_month_1.set_index(["pcod_from", "pcod_to", "date"])
    .unstack(fill_value=0)
    .stack()
    .reset_index()
)

trips_median_per_flow = (
    trips_subs_month_1_filled.groupby(["pcod_from", "pcod_to"])
    .value.median()
    .reset_index()
)

# Drop flows with median = 0 (we can't use these for scaling)
trips_median_per_flow = trips_median_per_flow.loc[trips_median_per_flow["value"] > 0]

### Trips subscribers month X 

In [ ]:
trips_days = []

for date in month_X_dates:
    with (
        Path(aggregates_dir)
        / "presence_trips"
        / f"presence_trips_aggregates_{data_date.date()}"
        / f"consecutive-trips_nosubset_{date.date()}.csv"
    ) as filepath:
        if Path(filepath).is_file():
            trips_day_X = pd.read_csv(
                Path(aggregates_dir)
                / "presence_trips"
                / f"presence_trips_aggregates_{data_date.date()}"
                / f"consecutive-trips_nosubset_{date.date()}.csv"
            )
            trips_day_X["date"] = date.date()

            trips_days.append(trips_day_X)

In [ ]:
trips_subs_month_X = pd.concat(trips_days)

### Static population estimates

In [ ]:
base_pop_and_growth_rates = pd.read_csv(base_pop_and_growth_rates_filepath).set_index(
    pop_and_gr_spatial_unit
)

base_pop = base_pop_and_growth_rates[base_pop_column]

### Scaling X factors

In [ ]:
x_scaling_factors = pd.read_csv(x_scaling_factors_filepath)


---

Now that we have read everything we need, create scaled base indicators for this month: 


---

# Presence

## Base indicators

In [ ]:
presence_month_x_intermediate = (
    presence_subs_month_X.merge(
        base_pop,
        on="pcod",
    )
    .merge(
        x_scaling_factors,
        on="pcod",
    )
    .merge(
        presence_median_per_loc,
        on="pcod",
        suffixes=("_presence", "_median_presence"),
    )
)

In [ ]:
presence_month_x_intermediate.head()

In [ ]:
presence_month_x_intermediate["presence"] = np.round(
    presence_month_x_intermediate[base_pop_column]
    + (
        presence_month_x_intermediate.scaling_factor
        * (
            presence_month_x_intermediate[base_pop_column]
            / presence_month_x_intermediate.value_median_presence
        )
        * (
            presence_month_x_intermediate.value_presence
            - presence_month_x_intermediate.value_median_presence
        )
    )
)

In [ ]:
presence_base = presence_month_x_intermediate[["pcod", "date", "presence"]]

# Trips in / out

In [ ]:
# Read in unredacted trips aggregates so we can sum over origins/destinations
trips_days_unredacted = []

for date in month_X_dates:
    with (
        Path(aggregates_dir)
        / "presence_trips"
        / f"presence_trips_aggregates_{data_date.date()}"
        / f"consecutive-trips_nosubset_{date.date()}_unredacted.csv"
    ) as filepath:
        if Path(filepath).is_file():
            trips_day_X_unredacted = pd.read_csv(
                Path(aggregates_dir)
                / "presence_trips"
                / f"presence_trips_aggregates_{data_date.date()}"
                / f"consecutive-trips_nosubset_{date.date()}_unredacted.csv"
            )
            trips_day_X_unredacted["date"] = date.date()

            trips_days_unredacted.append(trips_day_X_unredacted)

In [ ]:
trips_subs_month_X_unredacted = pd.concat(trips_days_unredacted)

In [ ]:
TRIPS_IN_subs = (
    trips_subs_month_X_unredacted[
        trips_subs_month_X_unredacted["pcod_from"]
        != trips_subs_month_X_unredacted["pcod_to"]
    ]
    .groupby(["pcod_to", "date"])
    .value.sum()
    .reset_index()
).rename(columns={"pcod_to": "pcod"})

# Redact after summing
TRIPS_IN_subs = TRIPS_IN_subs.loc[TRIPS_IN_subs["value"] > 15]

In [ ]:
TRIPS_OUT_subs = (
    trips_subs_month_X_unredacted[
        trips_subs_month_X_unredacted["pcod_from"]
        != trips_subs_month_X_unredacted["pcod_to"]
    ]
    .groupby(["pcod_from", "date"])
    .value.sum()
    .reset_index()
).rename(columns={"pcod_from": "pcod"})

# Redact after summing
TRIPS_OUT_subs = TRIPS_OUT_subs.loc[TRIPS_OUT_subs["value"] > 15]

In [ ]:
TRIPS_IN_subs = (
    TRIPS_IN_subs.merge(
        base_pop,
        on="pcod",
    )
    .merge(
        x_scaling_factors,
        on="pcod",
    )
    .merge(
        presence_median_per_loc,
        on="pcod",
        suffixes=("_trips", "_median_presence"),
    )
)

In [ ]:
TRIPS_OUT_subs = (
    TRIPS_OUT_subs.merge(
        base_pop,
        on="pcod",
    )
    .merge(
        x_scaling_factors,
        on="pcod",
    )
    .merge(
        presence_median_per_loc,
        on="pcod",
        suffixes=("_trips", "_median_presence"),
    )
)

In [ ]:
TRIPS_IN_people = (
    TRIPS_IN_subs.scaling_factor
    * (TRIPS_IN_subs[base_pop_column] / TRIPS_IN_subs.value_median_presence)
    * TRIPS_IN_subs.value_trips
)

In [ ]:
TRIPS_OUT_people = (
    TRIPS_OUT_subs.scaling_factor
    * (TRIPS_OUT_subs[base_pop_column] / TRIPS_OUT_subs.value_median_presence)
    * TRIPS_OUT_subs.value_trips
)

# Trips

## Base indicators

In [ ]:
trips_month_x_intermediate = (
    trips_subs_month_X.merge(
        presence_median_per_loc,
        left_on="pcod_to",
        right_on="pcod",
        suffixes=("_trips", "_median_presence"),
    )
    .merge(
        x_scaling_factors,
        left_on="pcod_to",
        right_on="pcod",
    )
    .merge(
        base_pop,
        left_on="pcod_to",
        right_on="pcod",
    )
)


trips_month_x_intermediate.head()

In [ ]:
trips_month_x_intermediate["travellers"] = np.round(
    trips_month_x_intermediate.scaling_factor
    * (
        trips_month_x_intermediate[base_pop_column]
        / trips_month_x_intermediate.value_median_presence
    )
    * (trips_month_x_intermediate.value_trips)
)

In [ ]:
trips_base = trips_month_x_intermediate[["pcod_from", "pcod_to", "date", "travellers"]]


---

Now that we have base indicators, create derived indicators for this month: 

## Indicators we need for derived indicators

### presence_people_month_1

In [ ]:
with Path(
    indicators_dir
) / f'presence_indicators_{presence_reference_date.strftime("%Y-%m")}.csv' as presence_fp:
    if (presence_fp).is_file():
        presence_people_month_1 = pd.read_csv(presence_fp)

    else:  # if they don't exist we are on month 1
        presence_people_month_1 = presence_base.copy()

In [ ]:
first_presence_people_month_1 = (
    presence_people_month_1.sort_values("date").groupby("pcod").presence.first()
)

In [ ]:
presence_intermediate = presence_base.merge(
    first_presence_people_month_1, on=["pcod"], suffixes=("", "_month_1_first")
)

### trips_people_month_1

In [ ]:
with Path(
    indicators_dir
) / f'movements_indicators_{trips_reference_date.strftime("%Y-%m")}.csv' as trips_fp:
    if (trips_fp).is_file():
        trips_people_month_1 = pd.read_csv(trips_fp)
    else:  # if they don't exist we are on month 1
        trips_people_month_1 = trips_base.copy()

In [ ]:
first_trips_people_month_1 = (
    trips_people_month_1.sort_values("date")
    .groupby(["pcod_from", "pcod_to"])
    .travellers.first()
)

In [ ]:
trips_intermediate = trips_base.merge(
    first_trips_people_month_1,
    on=["pcod_from", "pcod_to"],
    suffixes=("", "_month_1_median"),
)

### trips_people_month_X-1, X-2, X-3, X-4, X-5, X-6, ... X-12

In [ ]:
prev_6_mo = [
    (data_date - pd.DateOffset(months=i)).strftime("%Y-%m") for i in range(12, 0, -1)
]

months_data = 0

trips_prev_6_mo_list = []
for month in prev_6_mo:
    with Path(indicators_dir) / f"movements_indicators_{month}.csv" as trips_fp:
        if (trips_fp).is_file():
            months_data += 1
            trips_prev_6_mo_list.append(pd.read_csv(trips_fp))

if months_data == 0:
    trips_prev_6_mo_list.append(trips_people_month_1)

trips_prev_6_mo = (
    pd.concat(trips_prev_6_mo_list)
    .groupby(["pcod_from", "pcod_to"])
    .travellers.apply(lambda x: np.array(x))
)

In [ ]:
# need to pad the arrays so we include days with no data as 0's (as this will affect the abnormality)

In [ ]:
trips_prev_6_mo_max_days = trips_prev_6_mo.apply(len).max()

In [ ]:
trips_prev_6_mo = trips_prev_6_mo.apply(
    lambda z: np.pad(z, (0, trips_prev_6_mo_max_days - len(z)), "constant")
)

In [ ]:
trips_intermediate = trips_intermediate.merge(
    trips_prev_6_mo, on=["pcod_to", "pcod_from"], suffixes=("", "_last_6_mo")
)

### presence_people_month_X-1, X-2, X-3, X-4, X-5, X-6, ...,  X-12

In [ ]:
prev_6_mo = [
    (data_date - pd.DateOffset(months=i)).strftime("%Y-%m") for i in range(12, 0, -1)
]

months_data = 0
presence_prev_6_mo_list = []
for month in prev_6_mo:
    with Path(indicators_dir) / f"presence_indicators_{month}.csv" as presence_fp:
        if (presence_fp).is_file():
            months_data += 1
            presence_prev_6_mo_list.append(pd.read_csv(presence_fp))

if months_data == 0:
    presence_prev_6_mo_list.append(presence_people_month_1)

presence_prev_6_mo = (
    pd.concat(presence_prev_6_mo_list)
    .groupby("pcod")
    .presence.apply(lambda x: np.array(x))
)

In [ ]:
# need to pad the arrays so we include days with no data as 0's (as this will affect the abnormality)

In [ ]:
presence_prev_6_mo_max_days = presence_prev_6_mo.apply(len).max()

In [ ]:
presence_prev_6_mo = presence_prev_6_mo.apply(
    lambda z: np.pad(z, (0, presence_prev_6_mo_max_days - len(z)), "constant")
)

In [ ]:
presence_intermediate = presence_intermediate.merge(
    presence_prev_6_mo, on="pcod", suffixes=("", "_last_6_mo")
)

### Admin3 sizes

In [ ]:
admin3km2 = (
    gpd.read_file(geometry_filepath)
    .set_index("ADM3_PCODE")
    .to_crs(epsg=metric_crs_epsg)
    .area
    * 1e-6
)
admin3km2.name = "admin3_area_km2"

# Derived indicators (functions)

In [ ]:
def pct_change_with_ref(
    df,
    column,
    month_1_column,
    suffix="",
):
    return (
        (100 * (df[column] - df[month_1_column]) / df[month_1_column])
        .rename("relocations_pctchangewithref" + suffix)
        .to_frame()
    )

In [ ]:
def diff_with_ref(
    df,
    column,
    month_1_column,
    suffix="",
):
    return (
        (df[column] - df[month_1_column])
        .rename("relocations_pctchangewithref" + suffix)
        .to_frame()
    )

In [ ]:
def _mad(baseline: np.array):
    if (
        len(baseline) < 90
    ):  # only show abnormality when we have at least 90 days of data
        return np.nan
    else:
        return np.median(np.abs(baseline - np.median(baseline)))


def _meanad(baseline: np.array):
    if len(baseline) < 90:
        return np.nan
    else:
        return np.mean(np.abs(baseline - np.mean(baseline)))


def _mzscore(value, mad, meanad, median):
    if mad != 0:
        abnormality = (value - median) / (1.4826 * mad)
    elif meanad != 0:
        abnormality = (value - median) / (1.253314 * meanad)
    else:
        abnormality = np.nan
    return abnormality

In [ ]:
def abnorm(
    df,
    column,
    past_6_months_column,
    suffix="",
):
    mad = df[past_6_months_column].apply(lambda z: _mad(z)).rename("mad")
    meanad = df[past_6_months_column].apply(lambda z: _meanad(z)).rename("meanad")
    median = df[past_6_months_column].apply(lambda z: np.median(z)).rename("median")

    abnormality_intermediate = pd.concat(
        [
            mad,
            meanad,
            median,
            df[column],
        ],
        axis=1,
    )

    abnormality = (
        abnormality_intermediate.apply(
            lambda z: _mzscore(z[column], z["mad"], z["meanad"], z["median"]),
            axis=1,
        )
        .rename("abnormality" + suffix)
        .to_frame()
    )
    return abnormality

# Derived indicators (presence)

In [ ]:
#### pctchange

In [ ]:
presence_intermediate["presence_pctchangewithref"] = pct_change_with_ref(
    presence_intermediate, "presence", "presence_month_1_first"
)

In [ ]:
#### diffwref

In [ ]:
presence_intermediate["presence_diffwithref"] = diff_with_ref(
    presence_intermediate, "presence", "presence_month_1_first"
)

In [ ]:
#### abnormality

In [ ]:
presence_intermediate["abnormality"] = abnorm(
    presence_intermediate,
    "presence",
    "presence_last_6_mo",
    suffix="",
)

In [ ]:
#### trips_in

In [ ]:
presence_intermediate["trips_in"] = TRIPS_IN_people

In [ ]:
#### trips_out

In [ ]:
presence_intermediate["trips_out"] = TRIPS_OUT_people

In [ ]:
#### presence_km2

In [ ]:
presence_intermediate = presence_intermediate.merge(
    admin3km2, left_on="pcod", right_on="ADM3_PCODE"
)
presence_intermediate["presence_perKm2"] = (
    presence_intermediate["presence"] / presence_intermediate["admin3_area_km2"]
)

In [ ]:
####

In [ ]:
presence = presence_intermediate[
    [
        "date",
        "pcod",
        "presence",
        "presence_perKm2",
        "trips_in",
        "trips_out",
        "abnormality",
        "presence_diffwithref",
        "presence_pctchangewithref",
    ]
]

presence.to_csv(
    Path(indicators_dir) / f'presence_indicators_{data_date.strftime("%Y-%m")}.csv',
    index=False,
)

# Derived indicators (trips)

In [ ]:
#### pctchangewref

In [ ]:
trips_intermediate["travellers_pctchangewithref"] = pct_change_with_ref(
    trips_intermediate, "travellers", "travellers_month_1_median"
)

In [ ]:
#### diffwref

In [ ]:
trips_intermediate["travellers_diffwithref"] = diff_with_ref(
    trips_intermediate, "travellers", "travellers_month_1_median"
)

In [ ]:
#### abnormality

In [ ]:
trips_intermediate["abnormality"] = abnorm(
    trips_intermediate, "travellers", "travellers_last_6_mo"
)

In [ ]:
####

In [ ]:
trips = trips_intermediate[
    [
        "date",
        "pcod_from",
        "pcod_to",
        "travellers",
        "abnormality",
        "travellers_diffwithref",
        "travellers_pctchangewithref",
    ]
]

In [ ]:
trips.to_csv(
    Path(indicators_dir) / f'movements_indicators_{data_date.strftime("%Y-%m")}.csv',
    index=False,
)